# Autoencoder Inference Notebook

This notebook loads the trained autoencoder, applies it to test data, visualizes reconstructions, and prepares submission.

In [ ]:
info ="""

Author: Annam.ai IIT Ropar
Team Name: SoilClassifiers
Team Members: Caleb Chandrasekar, Sarvesh Chandran, Swaraj Bhattacharjee, Karan Singh, Saatvik Tyagi
Leaderboard Rank: 103

"""

In [ ]:
# inference.py

import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ✅ Environment
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ Configuration
IMG_SIZE   = 224
BATCH_SIZE = 1

# ✅ Custom Dataset
class SoilDataset(Dataset):
    def __init__(self, folder, transform=None):
        self.folder      = folder
        self.image_files = [f for f in os.listdir(folder)
                            if f.lower().endswith(('.png','jpg','jpeg'))]
        self.transform   = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.folder, img_name)
        img      = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, img_name

# ✅ Transforms (same as training)
test_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# ✅ Autoencoder (must match training.py)
class Autoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3,  32, 3, stride=2, padding=1), nn.ReLU(),
            nn.Conv2d(32, 64, 3, stride=2, padding=1), nn.ReLU(),
            nn.Conv2d(64,128, 3, stride=2, padding=1), nn.ReLU(),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(128, 64, 3, stride=2, padding=1, output_padding=1), nn.ReLU(),
            nn.ConvTranspose2d(64,  32, 3, stride=2, padding=1, output_padding=1), nn.ReLU(),
            nn.ConvTranspose2d(32,   3, 3, stride=2, padding=1, output_padding=1), nn.Sigmoid(),
        )

    def forward(self, x):
        return self.decoder(self.encoder(x))

# ✅ Load trained model & threshold
model     = Autoencoder().to(device)
model.load_state_dict(torch.load("autoencoder.pth", map_location=device))
model.eval()

threshold = float(np.load("threshold.npy"))
print(f"Loaded threshold: {threshold:.6f}")

# ✅ Prepare test loader
test_ds = SoilDataset(
    "/kaggle/input/soil-classification-part-2/soil_competition-2025/test",
    transform=test_transforms
)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)

# ✅ Inference loop
results = []
with torch.no_grad():
    for imgs, ids in test_loader:
        imgs  = imgs.to(device)
        recon = model(imgs)
        loss  = nn.MSELoss()(recon, imgs).item()
        label = 1 if loss < threshold else 0
        results.append((ids[0], label))

# ✅ Visualization helper
def show_reconstruction(orig, recon):
    orig = orig.cpu().squeeze().permute(1,2,0).numpy()
    recon = recon.cpu().squeeze().permute(1,2,0).numpy()
    fig, axes = plt.subplots(1,2, figsize=(6,3))
    axes[0].imshow((orig * 0.229 + 0.485).clip(0,1))
    axes[0].set_title("Original")
    axes[1].imshow((recon * 0.229 + 0.485).clip(0,1))
    axes[1].set_title("Reconstruction")
    plt.show()

# Show first 5 reconstructions
with torch.no_grad():
    for i, (imgs, _) in enumerate(test_loader):
        if i >= 5: break
        recon = model(imgs.to(device))
        show_reconstruction(imgs[0], recon[0])

# ✅ Save submission
submission = pd.DataFrame(results, columns=["image_id","label"])
submission.to_csv("submission.csv", index=False)
print("✅ submission.csv generated")
